In [ ]:
# This is the Python code I used for Metis Data Science Bootcamp Project 2
# Some comments have been added and some unnecessary code blocks were removed

In [ ]:
# This code block imports Python libraries

from __future__ import print_function
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from collections import defaultdict
from bs4 import BeautifulSoup
import requests
from IPython.core.display import display, HTML
import pickle
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import os
import time

In [ ]:
# This code block adjusts DataFrame display settings

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)
pd.set_option('display.precision', 6)

In [ ]:
# This code block uses the '.get' functionality from the requests library to access the target website
# A response status code of '200' indicates a successful API request

S_and_P_500_URL = 'https://www.slickcharts.com/sp500'
response = requests.get(S_and_P_500_URL)
#response.status_code # To view the response status, remove the left most hashtag from this line

In [ ]:
# This code block uses the web-scraping functionality BeautifulSoup (from the bs4 library)
# This code pulls the HTML from the website and accessed in the previous block and saves it as a bs4 object

S_and_P_page = response.text
S_and_P_soup = BeautifulSoup(S_and_P_page)
#S_and_P_soup # To view the saved bs4 object, remove the left most hashtag from this line

In [ ]:
# This code block creates a blank Pandas DataFrame for the Name, Symbol, and Weight of each company
# This DataFrame will be populated by a row of data for each company listed in the S&P 500

S_and_P_DF = pd.DataFrame(columns = ('Company', 'Symbol', 'Weight'))
#display(S_and_P_DF) # To view blank DataFrame, remove the left most hashtag from this line

In [ ]:
# This code block uses the '.find_all' functionality from bs4 to make a list using the table on the website
# Each row from the table (denoted by 'tr' in the HTML) will be one item in the List_of_Rows
# The first row (index number 0 on the list) is the header row with column names
# Each subsequent row had the data for one S & P company listing

List_of_Rows = list(S_and_P_soup.find_all('tr'))

#List_of_Rows # To view the list of rows, remove the left most hashtag from this line
#List_of_Rows[1] # To view a single example item from this list, remove the left most hashtag from this line

In [ ]:
# This code block creates a list with the most recent closing prices 
# Note: closing prices will differ from those used in my analysis, since you are running this on a different date

List_of_Closings = []

Index = 1 # Index initialized at 1 since index 0 refers to the header row, company data starts at 1

while(Index < len(List_of_Rows)):
    
    Row = str(List_of_Rows[Index]) # Creates a string for a single company's data
    
    TD_List = list(Row.split('/td')) # Each column of data had a seperate <td></td> pair of tags
    
    Price_String = TD_List[4] # The Closing Price is in the 5th column (index number 4 on the TD_List)
    #print(Price_String)
    
    Sub_Index = (len(Price_String)-1) # Price is near the end of the string with
    
    while(Price_String[Sub_Index] != " "): # These while loops find the indices needed to isolate the price data
        Sub_Index -= 1
    Sub_Index += 1
    End_Index = Sub_Index
    while(Price_String[End_Index] != "<"): 
        End_Index += 1
    List_of_Closings.append((Price_String[Sub_Index:End_Index]).strip("\xa0"))
    # The "\xa0" characters are an artifact of converting from HTML to bs4 to test strings
    
    Index += 1
    
#List_of_Closings # To view the list of closing prices, remove the left most hashtag from this line

In [ ]:
# This code block creates and populates lists for Company Name, S&P Symbol, and S&P Weight
# These lists will be used to populate the corresp

List_of_Companies = []
List_of_Symbols = []
List_of_Weights = []

Index = 1

while(Index < len(List_of_Rows)):
    
    Row = str(List_of_Rows[Index]) # Isolates a single company's data row
    
    # Initial Sub_Index value, and later incrementations, chosen to work for this specific site's HTML layout
    Sub_Index = 13 
    
    # These Boolean variables track whether a data item for the current row has been added to its respective list
    Company_Appended = False 
    Symbol_Appended = False
    Weight_Appended = False
    
    while(Company_Appended == False): # Loop to append Company Name
        
        if(Row[Sub_Index:(Sub_Index+7)] == '<a href'):
            Sub_Index += 18
            while(Row[Sub_Index] != '>'):
                Sub_Index += 1
            Sub_Index += 1
            Ending_Index = Sub_Index
            while(Row[Ending_Index] != '<'):
                Ending_Index += 1
            List_of_Companies.append(str(Row[Sub_Index:Ending_Index]))
            Company_Appended = True
        else:
            Sub_Index += 1
            
    Sub_Index += 13
            
    while(Symbol_Appended == False): # Loop to append Company Symbol
        
        if(Row[Sub_Index:(Sub_Index+7)] == '<a href'):
            Sub_Index += 18
            while(Row[Sub_Index] != '>'):
                Sub_Index += 1
            Sub_Index += 1
            Ending_Index = Sub_Index
            while(Row[Ending_Index] != '<'):
                Ending_Index += 1
            List_of_Symbols.append(str(Row[Sub_Index:Ending_Index]))
            Symbol_Appended = True
        else:
            Sub_Index += 1
            
    Sub_Index += 9
    
    while(Weight_Appended == False): # Loop to append Company Weight
        
        if(Row[Sub_Index:(Sub_Index+4)] == '<td>'):
            Sub_Index += 4
            Ending_Index = Sub_Index
            while(Row[Ending_Index] != '<'):
                Ending_Index += 1
            List_of_Weights.append(float(Row[Sub_Index:Ending_Index]))
            Weight_Appended = True
        else:
            Sub_Index += 1
            
    Index += 1

In [ ]:
# This code block uses the lists to populate the DataFrame columns for Company Name, S&P Symbol, and S&P Weight

S_and_P_DF['Company'] = List_of_Companies
S_and_P_DF['Symbol'] = List_of_Symbols
S_and_P_DF['Weight'] = List_of_Weights

#S_and_P_DF # To view this DataFrame remove the left most hashtag from this line

In [ ]:
# This code block creates a list of Yahoo Finance URLs that can be used for company-specific API calls
# The data on the Yahoo site contains a historical record of price changes, not just the most recent close price
# Constructing each company's URL will require inserting the company's ticker symbol at the correct point

List_of_Yahoo_Finance_History_URLs = []

for symbol in List_of_Symbols:
    
    Historical_Data_URL = 'https://finance.yahoo.com/quote/' + str(symbol) + '/history?p=' + str(symbol)
    List_of_Yahoo_Finance_History_URLs.append(Historical_Data_URL)
    
# Two listings (Berkshire and Brown Forman) have non-matching ticker symbols between the list and Yahoo Finance
# The code below corrects them by replacing a "." with a "-" in the symbol part of the URL

Berkshire_Index = (List_of_Yahoo_Finance_History_URLs.index('https://finance.yahoo.com/quote/BRK.B/history?p=BRK.B'))
Brown_Forman_Index = (List_of_Yahoo_Finance_History_URLs.index('https://finance.yahoo.com/quote/BF.B/history?p=BF.B'))

List_of_Yahoo_Finance_History_URLs[Berkshire_Index] = 'https://finance.yahoo.com/quote/BRK-B/history?p=BRK-B'
List_of_Yahoo_Finance_History_URLs[Brown_Forman_Index] = 'https://finance.yahoo.com/quote/BF-B/history?p=BF-B'

#List_of_Yahoo_Finance_History_URLs # To view the list of URLs remove the left most hashtag from this line

In [ ]:
# This code block scapes closing price and adjusted closing price data from a selected historical date
# Note: The date I originally used ifor this part of the project was 'Sep 05, 2019'
# There is a limit to how far back one can scrape with this site and it was Sept. 5 2019 at the time
# Further Note: Two companies (Otis Worldwide (OTIS) and Carrier Global (CARR)) were later added to the S&P
# The Yahoo Finance pages for these 2 new companies only have stock price data going back to March 19, 2020

Date_To_Scrape_From = "Mar 19, 2020" 
# In this example, March 19, 2020 was used so every entry (including recent additions) will have a closing price
# Alternatively, a user can choose to go further back and assign NaN values to the few that don't have prices
# See the code for that below

List_of_Historic_Closing_Prices = []
List_of_Historic_Adjusted_Closing_Prices = []

URLs_Not_Appended = [] # List to check if a company failed to have data scraped (should be empty)
URLs_Multiple_Appended = [] # List to check if a company had multiple rows of data scraped (should be empty)

URL_Index = 0

Check_Interval = 20 # If you want to monitor the code blocks progress at regular intervals, you can adjust this
# This will determine the frequency of progress updates (eg: "20 complete", "40 complete", etc if interval = 20)

while(URL_Index < len(List_of_Yahoo_Finance_History_URLs)):

    URL = List_of_Yahoo_Finance_History_URLs[URL_Index]
    
    Added = False
    Appends = 0
    Attempted_APIs = 0

    while((Attempted_APIs < 5) and (Added == False)):
        
        response = requests.get(URL)
        if(response.status_code != 200):
            print(URL_Index, URL, "Failed API", response.status_code)    
        Historic_Data_Text = response.text
        Historic_Data_List = list((Historic_Data_Text).split("<tr"))
    
        for Row in Historic_Data_List:
            if(Date_To_Scrape_From in Row):
                if("Dividend" in Row):
                    pass
                else:
                    Span_List = list(Row.split("<span"))
                    Append_List = []
                    for Span in Span_List:
                        Index = 0
                        while(Span[Index] != ">"):
                            Index += 1
                        Index += 1
                        End_Index = Index
                        while(Span[End_Index] != "<"):
                            End_Index += 1
                        Append_List.append(Span[Index:End_Index])
                    if(len(Append_List) > 6):
                        List_of_Historic_Closing_Prices.append(Append_List[5])
                        List_of_Historic_Adjusted_Closing_Prices.append(Append_List[6])
                        Added = True
                        Appends += 1
                    else:
                        Added = True
                        List_of_Historic_Closing_Prices.append(np.nan)
                        List_of_Historic_Adjusted_Closing_Prices.append(np.nan)

        if(Added == False):
            print(URL_Index, URL, "Not Added")
            Attempted_APIs += 1
            time.sleep(5)

        if(Appends == 0):
            URLs_Not_Appended.append(URL)

        if(Appends > 1):
            URLs_Multiple_Appended.append(URL)
            
    if(Added == False):
        List_of_Historic_Closing_Prices.append(np.nan)
        List_of_Historic_Adjusted_Closing_Prices.append(np.nan)
        
    URL_Index += 1
    
    if((URL_Index % Check_Interval) == 0): # Un-comment-ify this if-block to get progress updates
        print(URL_Index, "complete")
        
print(len(List_of_Historic_Closing_Prices)) # To verify all companies had an entry added, check the list's length
# There should be 505 entries on this list (a few companies are doubly listed on the S&P 500)

In [ ]:
# You can use the code below to check if any URL API calls failed to get closing prices for the date

print(URLs_Not_Appended) # No closing price (np.nan appended for that entry)
print(URLs_Multiple_Appended) # Multiple closing prices

In [ ]:
# This code block updates the DataFrame with the closing prices from the relevant dates

S_and_P_DF['Mar 19, 2020 Close'] = List_of_Historic_Closing_Prices
S_and_P_DF['Mar 19, 2020 Adjusted Close'] = List_of_Historic_Adjusted_Closing_Prices
S_and_P_DF['Most Recent Closing Price'] = List_of_Closings

#display(S_and_P_DF) # To view the updated dataframe remove the hashtag in front of the display command

In [ ]:
# This code block defines the NoCommas(string) function
# This function helps convert string representations of prices/finance into float values if the string has commas

def NoCommas(string):
    
    Output = ""
    String_Index = 0
    
    while(String_Index < len(string)):
        if(string[String_Index] == ","):
            pass
        else:
            Output = Output + str(string[String_Index])
            
        String_Index += 1
        
    return Output

In [ ]:
# This code block converts string variables to float variables in the DataFrame

Index = 0

while(Index < len(S_and_P_DF)):
    
    if(type(S_and_P_DF['Most Recent Closing Price'][Index]) == str):
        S_and_P_DF['Most Recent Closing Price'][Index] = NoCommas(S_and_P_DF['Most Recent Closing Price'][Index])
        S_and_P_DF['Most Recent Closing Price'][Index] = float(S_and_P_DF['Most Recent Closing Price'][Index])
    
    if(type(S_and_P_DF['Mar 19, 2020 Close'][Index]) == str):
        S_and_P_DF['Mar 19, 2020 Close'][Index] = NoCommas(S_and_P_DF['Mar 19, 2020 Close'][Index])
        S_and_P_DF['Mar 19, 2020 Close'][Index] = float(S_and_P_DF['Mar 19, 2020 Close'][Index])
    
    if(type(S_and_P_DF['Mar 19, 2020 Adjusted Close'][Index]) == str):
        S_and_P_DF['Mar 19, 2020 Adjusted Close'][Index] = NoCommas(S_and_P_DF['Mar 19, 2020 Adjusted Close'][Index])
        S_and_P_DF['Mar 19, 2020 Adjusted Close'][Index] = float(S_and_P_DF['Mar 19, 2020 Adjusted Close'][Index])
    
    Index += 1
    
#display(S_and_P_DF) # To view the updated dataframe remove the hashtag in front of the display command

In [ ]:
# This code block calculates the percentage increase from the scraped historical date to the recent close
# The precent is calculated as a decimal (eg 20% == 0.20) but you can multiply by 100% to get normal percentage
# Stocks that declined in price during that time have a negative percent increase

Index = 0

List_of_Increases = []
List_of_Percent_Increases = []

while(Index < len(S_and_P_DF)):
    Inc = (float(S_and_P_DF['Most Recent Closing Price'][Index]) - float(S_and_P_DF['Mar 19, 2020 Adjusted Close'][Index]))
    Per = Inc/float(S_and_P_DF['Mar 19, 2020 Adjusted Close'][Index])
    
    List_of_Increases.append(Inc)
    List_of_Percent_Increases.append(Per)
    
    Index += 1

In [ ]:
# This code block adds the increase amount and percentage to the DataFrame

S_and_P_DF['Gross Increase'] = List_of_Increases
S_and_P_DF['Percent Increase'] = List_of_Percent_Increases

#display(S_and_P_DF) # To view the updated DataFrame remove the hashtag in front of the display command

In [ ]:
# This code block identifies creates a new DataFrame from a CSV file with data on each company's industry
# You must use the 'constituents_csv' CSV file for this block along with your own file path

Industry_DF = pd.read_csv('/Your_File_Path/constituents_csv.csv') # Replace with your file path

#display(Industry_DF)

In [ ]:
# This code block merges the two DataFrames by joining on symbol

S_and_P_With_Industry_DF = pd.DataFrame(S_and_P_DF.join(Industry_DF.set_index('Symbol'), on='Symbol'))

display(S_and_P_With_Industry_DF)

In [ ]:
# This code block fills in missing industry sector values with "Other"

S_and_P_With_Industry_DF['Sector'] = S_and_P_With_Industry_DF.Sector.fillna('Other')
    
#display(S_and_P_With_Industry_DF)

In [ ]:
# This code block checks for all unique industries values that populate the Sector Field

S_and_P_With_Industry_DF['Sector'].unique()

In [ ]:
# In order to create one-hot encoded dummy variables, this code block creates a list for each industry
# Each industry list will have an entry for each company with either a "1" (if in that sector) or a "0"

Uniques = ['Information Technology', 'Consumer Discretionary', 'Financials', 'Health Care', 'Consumer Staples',
           'Energy', 'Telecommunication Services', 'Industrials', 'Utilities','Real Estate', 'Materials']

Information_Tech_List = []
Consumer_Discretionary_List = []
Customer_Staples_List = []
Financials_List = []
Healthcare_List = []
Energy_List = []
Telecom_List = []
Industries_List = []
Utilities_List = []
Real_Estate_List = []
Materials_List = []

List_of_Lists = [Information_Tech_List, Consumer_Discretionary_List, Financials_List, Healthcare_List,
                 Customer_Staples_List, Energy_List, Telecom_List, Industries_List, Utilities_List,
                 Real_Estate_List, Materials_List]

Index = 0

while(Index < len(S_and_P_With_Industry_DF)):
    List_Index = 0
    Industry_Found = False
    while((Industry_Found == False) and (List_Index < len(Uniques))):
        if(S_and_P_With_Industry_DF['Sector'][Index] == Uniques[List_Index]):
            List_of_Lists[List_Index].append(1)
            Industry_Found = True
        else:
            List_of_Lists[List_Index].append(0)
        List_Index += 1
    while(List_Index < len(Uniques)):
        List_of_Lists[List_Index].append(0)
        List_Index += 1
    Index += 1

In [ ]:
# This code block creates and populates columns for the one-hot encoded sector variables

S_and_P_With_Industry_DF['Information Technology'] = Information_Tech_List
S_and_P_With_Industry_DF['Consumer Discretionary'] = Consumer_Discretionary_List
S_and_P_With_Industry_DF['Financials'] = Financials_List
S_and_P_With_Industry_DF['Health Care'] = Healthcare_List
S_and_P_With_Industry_DF['Consumer Staples'] = Customer_Staples_List
S_and_P_With_Industry_DF['Energy'] = Energy_List
S_and_P_With_Industry_DF['Telecommunication Services'] = Telecom_List
S_and_P_With_Industry_DF['Industrials'] = Industries_List
S_and_P_With_Industry_DF['Utilities'] = Utilities_List
S_and_P_With_Industry_DF['Real Estate'] = Real_Estate_List
S_and_P_With_Industry_DF['Materials'] = Materials_List

display(S_and_P_With_Industry_DF)

In [ ]:
# This code block scrapes financial data (Total Revenue, Gross Profit, Operating Expense, etc) for each company
# This block takes a while to run due to a need for a delay between API calls
# To keep track of progress, remove the "#"s from in front of the print statements

symbol = [] # Company symbol
name = [] # Name of field ("Total Revenue", "Gross Profit", etc)
value = [] # Value in field

for i, s in enumerate(List_of_Symbols):
    
    Financials_URL = 'https://finance.yahoo.com/quote/' + s + '/financials?p=' + s
    API_Attempts = 0
    Done_With_Company = False
    
    while((API_Attempts < 5) and (Done_With_Company == False)):
        
        response = requests.get(Financials_URL)
        
        if(response.status_code == 200):
            soup = BeautifulSoup(response.text, "html5lib")
            Fin_Elements = soup.find(class_='D(tbrg)')
            print(" ")
            print("--------------------------------------------------------------------------------")
            if Fin_Elements:
                print(" ")
                print(Financials_URL, "Success")
                print(" ")
                for rows in Fin_Elements.find_all(class_='D(tbr)'):
                    elems = rows.find_all(class_='D(tbc)')
                    symbol.append(s)
                    name.append(elems[0].text)
                    value.append(elems[1].text)
            else:
                print(" ")
                print(Financials_URL, "<--------- API Success, Scraping Fail:", response.status_code)    
                print(" ")
            Done_With_Company = True
        else:
            API_Attempts += 1
            print(" ")
            print("--------------------------------------------------------------------------------")
            print(" ")
            print(Financials_URL, "<--- Failed API Attempt:", API_Attempts, response.status_code)    
            print(" ") 
            time.sleep(3 ** (API_Attempts))
            
    if(Done_With_Company == False):
        print(" ")
        print("--------------------------------------------------------------------------------")
        print(" ")
        print(Financials_URL, "<-------- Failed  All API Attempts:", response.status_code)   
        print(" ")
        
    time.sleep(2) # This line imposes a delay between API calls to avoid crashing the website

In [ ]:
# This code block creates a DataFrams with company financil data, data category, and company symbol

Financials_DF = pd.DataFrame()
Financials_DF['Symbol'] = symbol
Financials_DF['Name'] = name
Financials_DF['Value'] = value

#display(Financials_DF)

In [ ]:
# This code block creates a new DataFrame with a column for each type of financial data
# The values in the 'name' column of the previous DataFrame become column names for the new DataFrame

Financials_Unstacked_DF = Financials_DF.groupby(['Symbol','Name']).max()
Financials_Unstacked_DF = Financials_Unstacked_DF.unstack(1)
Financials_Unstacked_DF.columns = Financials_Unstacked_DF.columns.droplevel()
Financials_Unstacked_DF['Symbol'] = Financials_Unstacked_DF.index

#display(Financials_Unstacked_DF)

In [ ]:
# This code block adjusts the symbol for Berkshire Hathaway

Financials_Unstacked_DF['Symbol']['BRK.B'] = 'BRK-B'

In [ ]:
# This code block allows the user to view which columns have been successfully populated and for how many entries

Financials_Unstacked_DF.info()

In [ ]:
# This code block merges the Finacials dataframe with the existing S&P dataframe to create the full data set

Full_S_and_P_DF = pd.DataFrame(S_and_P_With_Industry_DF.join(Financials_Unstacked_DF.set_index('Symbol'), on='Symbol'))

display(Full_S_and_P_DF)

In [ ]:
# This code block saves the DataFrame in the current folder as a pickle file 

with open('Full_S_and_P_DF.pkl', 'wb') as picklefile:
    pickle.dump(Full_S_and_P_DF, picklefile)